## 📄 Digital Brochure Generator

An automated solution designed to generate high-impact corporate brochures from minimal data. Simply provide the company name and website, and the tool creates informative material tailored to different strategic audiences.

#### 🚀 Project Description
This tool streamlines the creation of marketing and sales materials. Using the company website as the primary source, it extracts, synthesizes, and structures key information to generate customized brochures for three main profiles:
* **Potential Clients:** Focus on products, services, and value proposition.

* **Investors:** Focus on metrics, vision, market, and business stability.

* **Potential Candidates (Recruiting):** Focus on culture, benefits, and mission.

#### 🛠 Inputs and Outputs
* **Input:** Company name and website URL.
* **Output:** Structured digital brochure (PDF/Markdown).

In [132]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [133]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

API key looks good so far


In [ ]:
# List the available models
models = openai.models.list()

for model in models:
    print(model.id)

In [134]:
MODEL = "gpt-4.1-mini"
SITE = "HuggingFace"
URL = "https://huggingface.co"

In [135]:
# Create this Website object from the given url using the BeautifulSoup library
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to scrape and parse website content.
    """
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            # Remove irrelevant elements like scripts, styles, and images to reduce token usage
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            # Extract clean text
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        
        links = [link.get("href") for link in soup.find_all("a")]
        self.links = [link for link in links if link]
        
    def get_contents(self):
        return f"Website({self.url}, {self.title}, {self.text}, {self.links})"

#### First step: This is where the LLM decides which links are relevant to the brochure

In [ ]:
# System Prompt using One-shot Prompting
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.

You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [136]:
# System Prompt using Multi-shot (Few-Shot) Prompting
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, Company page, Careers/Jobs, or specific Product pages.

You should respond in JSON. Here are examples of how to handle different scenarios:

Example 1 (Corporate Site):
Input Links: ["/home", "/about-us", "/blog/article-1", "/careers", "/privacy-policy", "/contact"]
Response:
{
    "links": [
        {"type": "about page", "url": "https://company.com/about-us"},
        {"type": "careers page", "url": "https://company.com/careers"},
        {"type": "contact page", "url": "https://company.com/contact"}
    ]
}

Example 2 (E-commerce/Product Focus):
Input Links: ["/", "/shop/men", "/shop/women", "/terms", "/returns", "/our-story"]
Response:
{
    "links": [
        {"type": "about page", "url": "https://shop.com/our-story"},
        {"type": "product category", "url": "https://shop.com/shop/men"},
        {"type": "product category", "url": "https://shop.com/shop/women"}
    ]
}

Example 3 (Service Provider):
Input Links: ["/services/consulting", "/team", "/login", "/signup", "/legal"]
Response:
{
    "links": [
        {"type": "services page", "url": "https://service.com/services/consulting"},
        {"type": "team page", "url": "https://service.com/team"}
    ]
}
"""

In [137]:
def get_links_user_prompt(url):
    site = Website(url)
    user_prompt = f"""
    Here is the list of links on the website: {site.url}
    Please decide which of these are relevant web links for a brochure about the company,
    respond with the full https URL in JSON format.
    Do not include Terms of Service, Privacy, email links.
    
    Links (some might be relative links):
    {site.links}
    """
    return user_prompt

In [ ]:
site = Website(URL)
print(site.get_contents())
site.links

In [ ]:
print(get_links_user_prompt(URL))

In [139]:
def get_links(url):
    """
    Uses the LLM to analyze and filter relevant links from the website.
    """
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
get_links(URL)

#### Second step: Creating the brochure!

This section compiles the content from all selected pages and generates the final brochure

In [140]:
def get_all_details(url):
    """
    Aggregates content from the landing page and all identified relevant sub-pages.
    """
    result = "Landing Page:\n"
    result += Website(url).get_contents()
    result += "\n\nRelevant Links:\n"
    relevant_links = get_links(url)
    # Iterate through each relevant link, scrape it, and append its content
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details(URL))

In [126]:
# System prompt: Instructs the LLM to write the final marketing brochure
brochure_system_prompt = """
You are a top-tier corporate marketing consultant designed to create high-impact executive brochures.
Analyze the provided company website contents and generate a professional, polished brochure targeting prospective clients, investors, and top talent.

**Guidelines:**
- **Tone:** Formal, persuasive, corporate, and trustworthy. Avoid casual language; sound like a Fortune 500 company.
- **Format:** Clean Markdown (no code blocks).
- **Visuals:** You MUST use relevant professional emojis/icons (e.g., 🚀, 🏢, 💼, 🌟, 📈, 🤝) at the start of every section header and for key bullet points to enhance readability and visual appeal.
- **Structure:**
  - 🏢 **Company Overview:** Mission and vision.
  - 🚀 **Key Offerings:** Products/services and value proposition.
  - 📈 **For Investors:** Market position, metrics, and stability.
  - 🤝 **Culture & Careers:** Benefits and environment for prospective recruits.
"""

In [141]:
# System Prompt for Structured Corporate Brochure
brochure_system_prompt = """
You are a top-tier corporate marketing consultant designed to create high-impact executive brochures.
Analyze the provided company website contents and generate a professional brochure.

**Formatting Constraints:**
1. Use professional emojis (🚀, 🏢, 💡, 🤝) for headers.
2. Output strictly in Markdown.
3. Do not include code blocks.

**Required Sections Structure:**
You must organize the content into exactly these four sections:

1. 🏢 **Executive Summary & Mission**
   - Synthesize the company's core purpose and vision.
   
2. 🚀 **Key Solutions & Value Proposition**
   - Detail the main products or services.
   - Explain why customers choose them (USPs).

3. 📊 **Market & Investor Data**
   - Focus on growth, market position, and stability.
   - Mention partners or key clients if available.

4. 🤝 **Talent & Culture**
   - Describe the work environment and benefits for potential recruits.
"""

In [142]:
def get_brochure_user_prompt(company_name, url):
    # Build the prompt with the aggregated context from all pages
    user_prompt = f"""
    You are looking at a company called: {company_name}
    Here are the contents of its landing page and other relevant pages;
    use this information to build a short brochure of the company in markdown without code blocks.
    
    {get_all_details(url)}
    """
    # Truncate to avoid exceeding context window limits (simple safety check)
    return user_prompt[:50000]

In [143]:
def create_brochure(company_name, url):
    """
    Generates the brochure (non-streaming version).
    """
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [144]:
create_brochure(SITE, URL)

# Hugging Face: Shaping the Future of AI 🤖

---

🏢 **Executive Summary & Mission**

Hugging Face is the premier collaboration platform for the machine learning (ML) community, powering the future of open and ethical AI. As a central hub, Hugging Face enables engineers, scientists, and end-users worldwide to share, discover, and experiment with state-of-the-art open-source ML models, datasets, and applications. Fueled by a thriving global community and cutting-edge research, the company is at the heart of the ongoing AI revolution—empowering democratized AI development and accelerating innovation with transparency and inclusivity.

---

🚀 **Key Solutions & Value Proposition**

- **Hugging Face Hub:** A vast repository hosting over 1 million ML models and 250k+ datasets covering NLP, computer vision, audio, and more. Users can explore, use, and contribute to cutting-edge AI work easily.
- **Spaces:** A dynamic AI App Directory offering thousands of interactive artificial intelligence applications—ranging from image and video generation to speech synthesis, text analysis, and 3D modeling—all accessible in-browser with scalable compute options.
- **Open Source Libraries:** Industry-leading open-source toolkits such as Transformers (for NLP), Diffusers (for diffusion models), Tokenizers, PEFT (parameter-efficient fine-tuning), and more, driving innovation in AI research and production.
- **Enterprise Hub:** A secure, enterprise-grade platform with features like Single Sign-On, granular access control, private dataset viewers, advanced compute options, and priority support, designed to scale AI deployments for teams and organizations.
- **Inference Endpoints & Providers:** Seamlessly deploy and scale ML models with dedicated infrastructure and access to 45,000+ models from multiple inference partners through a unified API with no service fees.
- **Compute & Pricing Flexibility:** Pay-as-you-go and subscription plans tailored for individuals, teams, and enterprises, enabling access to premium hardware resources like Nvidia GPUs and zero-GPU quotas to optimize development workflows.

Why customers choose Hugging Face:  
They gain unparalleled access to the broadest machine learning ecosystem built for collaboration, rapid experimentation, and state-of-the-art model deployment—all backed by robust infrastructure, enterprise security, and an active community.

---

📊 **Market & Investor Data**

- Trusted and used by over 50,000 organizations globally, including leading AI innovators and enterprises such as Meta, Microsoft, Google, Amazon, NVIDIA, IBM, Grammarly, and OpenAI.
- Boasts a vibrant developer and researcher community with growing engagement metrics: millions of model downloads and inference requests each week.
- Enterprise clients benefit from GDPR compliance and SOC 2 Type 2 certifications, underscoring the commitment to security and compliance.
- Hugging Face’s open-source libraries like Transformers have amassed over 150,000 stars on GitHub, signifying widespread adoption and community trust.
- Continuously evolving platform with new model architectures and features spotlighted on their popular blog and community forums, fueling sustained growth and thought leadership in AI.

---

🤝 **Talent & Culture**

At Hugging Face, innovation is built on open collaboration, diversity, and empowerment. The company fosters a dynamic startup environment where talent from all over the globe work together to push the boundaries of AI technology.

- **Work Environment:** Remote-friendly, inclusive, and highly collaborative culture that encourages autonomy and continuous learning.
- **Career Opportunities:** Positions across engineering, research, product, and community roles supporting AI at scale. Candidates join a passionate team focused on ethical AI and open-source contributions.
- **Employee Benefits:** Competitive salaries, flexible work policies, access to cutting-edge tools, and opportunities to engage with the global ML community.
- **Community Engagement:** Employees actively contribute to open source, share research insights in public forums, and participate in AI education initiatives, reinforcing a culture of openness and impact-driven work.

Join Hugging Face to be part of the AI future—building technology that empowers millions and shapes the ethical landscape of machine learning worldwide.

---

For more information, visit [huggingface.co](https://huggingface.co) or explore career opportunities at [apply.workable.com/huggingface](https://apply.workable.com/huggingface/).

In [130]:
def stream_brochure(company_name, url):
    """
    Generates the brochure with streaming enabled for a better user experience.
    """
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True # Enable streaming
    )
    
    response = ""
    # Create a placeholder for the display
    display_handle = display(Markdown(""), display_id=True)
    
    # Update the display in real-time as chunks arrive
    for chunk in stream:
        if chunk.choices[0].delta.content:
            response += chunk.choices[0].delta.content
            update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure(SITE, URL)

#### Additional Step: Brochure Translation

This is the new logic. We need a function that takes the text generated in step 2 and makes an additional call to the LLM to translate it.

In [145]:
def translate_brochure(brochure_text, target_language="English"):
    """
    Translates the generated brochure into the target language using a fresh LLM call.
    """
    translation_system_prompt = f"""
    You are a professional translator specializing in corporate marketing materials.
    Translate the following markdown text into {target_language}.
    
    IMPORTANT:
    - Maintain all original Markdown formatting (headers, bolding, lists).
    - Keep all emojis/icons exactly as they are.
    - Ensure the tone remains professional and persuasive in the target language.
    """
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": translation_system_prompt},
            {"role": "user", "content": brochure_text}
        ],
    )
    
    translated_content = response.choices[0].message.content
    return translated_content

In [149]:
def create_brochure_text(company_name, url):
    """
    Generates the brochure text (without displaying it immediately) so we can pipeline it.
    """
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    return response.choices[0].message.content

In [150]:
# --- MAIN EXECUTION FLOW ---

# 1. Generate the original brochure (Spanish/Original Language)
print("Generating original brochure...")
original_brochure = create_brochure_text(SITE, URL)
display(Markdown("## 🇺🇸 Original Brochure"))
display(Markdown(original_brochure))

print("\n------------------------------------------------\n")

# 2. Translate the brochure (Third LLM Call)
print("Translating brochure to Spanish...")
english_brochure = translate_brochure(original_brochure, target_language="Spanish")
display(Markdown("## 🇪🇸 Translated Brochure (Spanish)"))
display(Markdown(english_brochure))

Generating original brochure...


## 🇺🇸 Original Brochure

# Hugging Face Corporate Brochure

---

## 🏢 Executive Summary & Mission

Hugging Face is the premier collaboration platform for the global machine learning (ML) community. As the home of open-source ML innovation, Hugging Face empowers engineers, scientists, and AI practitioners to create, discover, and share transformative AI technologies. Its vision is to build an open, ethical, and collaborative AI future where cutting-edge models, datasets, and applications are accessible to all. With a rapidly expanding user base and a strong commitment to community-driven development, Hugging Face is at the heart of the AI revolution.

---

## 🚀 Key Solutions & Value Proposition

- **Comprehensive AI Hub:** Access over 1 million pre-trained models spanning text, images, audio, video, and even 3D modalities. Hugging Face provides seamless discovery, experimentation, and deployment within a single unified platform.

- **Datasets & Spaces:** Access 250k+ diverse datasets and share interactive ML applications through Spaces — an AI app directory featuring cutting-edge demos and tools for image and video generation, text analytics, speech synthesis, and more.

- **Open-Source Libraries:** Leading open-source tools including Transformers, Diffusers, Tokenizers, and Accelerate empower developers with state-of-the-art research-grade AI models optimized for production.

- **Enterprise-Grade AI Platform:** Scalable solutions starting at $20/user/month with advanced security features like single sign-on (SSO), audit logs, granular permissions, private dataset viewers, and priority support. Enterprise clients enjoy custom onboarding, managed billing, and compliance certifications including GDPR and SOC 2 Type 2.

- **Unified API & Compute:** Connect to 45,000+ models from top AI providers via one API with no service fees. Deploy with flexible compute options including GPU-powered inference endpoints, optimized for performance and scale.

Why Customers Choose Hugging Face:
- A vibrant, rapidly growing AI community fostering innovation  
- Unmatched breadth and depth of ML resources across modalities and tasks  
- Enterprise-grade security and support tailored for organizations of all sizes  
- Continuous integration of newest research breakthroughs into easy-to-use products  
- Transparent pricing and accessible compute infrastructure for teams and individuals  

---

## 📊 Market & Investor Data

- Trusted by **50,000+ organizations** worldwide, including AI leaders and Fortune 500 companies such as **Google, Microsoft, Amazon, Meta, NVIDIA, IBM, Grammarly, and Toyota Research Institute**.

- Hosts an extensive portfolio of over **45,000 models** from prominent AI teams and providers, accompanied by vibrant community contributions and user engagement metrics in the hundreds of thousands.

- Open-source libraries like Transformers have been downloaded and starred extensively on GitHub, forming the backbone of many commercial and academic AI innovations.

- Enterprise clients benefit from flexible subscription tiers — Team ($20+/user/month) and Enterprise (custom pricing), ensuring scalable adoption with industry-standard security and compliance.

- Hugging Face maintains strong partnerships with cloud infrastructure providers and advances AI compute efficiency through GPU inference endpoints and ZeroGPU hosting.

---

## 🤝 Talent & Culture

Hugging Face fosters a dynamic, inclusive, and innovation-driven work environment committed to building the future of machine learning. By joining their team, you become part of a mission to democratize AI while engaging with some of the most talented minds in the field.

- **Learning and Growth:** Access to cutting-edge technologies and direct collaboration with the open-source AI community.

- **Flexibility:** Roles that support remote work and flexible schedules, empowering employees to balance productivity with personal well-being.

- **Impact:** Opportunity to influence global AI development in a transparent, community-first company.

- **Benefits:** Competitive compensation, comprehensive health coverage, equity options, and professional development support.

- **Collaborative Culture:** Cross-functional teams working openly with a bias towards experimentation, learning, and ethical use of AI.

Hugging Face actively recruits top talent passionate about AI and open collaboration. Current openings and application details are available at https://apply.workable.com/huggingface/.

---

*For press inquiries or partnerships, contact press@huggingface.co*  
*Discover more & join the AI revolution at [huggingface.co](https://huggingface.co)*


------------------------------------------------

Translating brochure to Spanish...


## 🇪🇸 Translated Brochure (Spanish)

# Folleto Corporativo de Hugging Face

---

## 🏢 Resumen Ejecutivo y Misión

Hugging Face es la plataforma de colaboración líder para la comunidad global de aprendizaje automático (ML). Como el hogar de la innovación de ML de código abierto, Hugging Face empodera a ingenieros, científicos y profesionales de IA para crear, descubrir y compartir tecnologías de IA transformadoras. Su visión es construir un futuro de IA abierto, ético y colaborativo, donde modelos, conjuntos de datos y aplicaciones de vanguardia sean accesibles para todos. Con una base de usuarios en rápida expansión y un fuerte compromiso con el desarrollo impulsado por la comunidad, Hugging Face está en el corazón de la revolución de la IA.

---

## 🚀 Soluciones Clave y Propuesta de Valor

- **Hub Integral de IA:** Acceso a más de 1 millón de modelos preentrenados que abarcan texto, imágenes, audio, video e incluso modalidades 3D. Hugging Face ofrece un descubrimiento, experimentación y despliegue sin interrupciones dentro de una plataforma unificada.

- **Conjuntos de Datos y Espacios:** Acceso a más de 250,000 conjuntos de datos diversos y la posibilidad de compartir aplicaciones interactivas de ML a través de Spaces — un directorio de aplicaciones de IA con demostraciones y herramientas de punta para generación de imágenes y videos, análisis de texto, síntesis de voz y más.

- **Librerías de Código Abierto:** Herramientas de código abierto líderes como Transformers, Diffusers, Tokenizers y Accelerate que capacitan a los desarrolladores con modelos de IA de nivel investigación, optimizados para producción.

- **Plataforma de IA para Empresas:** Soluciones escalables desde $20/usuario/mes con características avanzadas de seguridad como inicio de sesión único (SSO), registros de auditoría, permisos granulares, visualizadores privados de conjuntos de datos y soporte prioritario. Los clientes empresariales disfrutan de onboarding personalizado, facturación gestionada y certificaciones de cumplimiento, incluyendo GDPR y SOC 2 Tipo 2.

- **API Unificada y Computación:** Conexión a más de 45,000 modelos de los principales proveedores de IA mediante una sola API sin cargos por servicio. Despliegue con opciones de cómputo flexibles que incluyen endpoints de inferencia impulsados por GPU, optimizados para rendimiento y escala.

Por qué los clientes eligen Hugging Face:  
- Una comunidad de IA vibrante y en rápido crecimiento que fomenta la innovación  
- Alcance y profundidad inigualables de recursos de ML en múltiples modalidades y tareas  
- Seguridad y soporte de nivel empresarial diseñados para organizaciones de todos los tamaños  
- Integración continua de los últimos avances en investigación en productos fáciles de usar  
- Precios transparentes e infraestructura accesible de cómputo para equipos e individuos  

---

## 📊 Mercado y Datos para Inversionistas

- Confiado por más de **50,000 organizaciones** en todo el mundo, incluyendo líderes en IA y empresas Fortune 500 como **Google, Microsoft, Amazon, Meta, NVIDIA, IBM, Grammarly y Toyota Research Institute**.

- Alberga un extenso portafolio de más de **45,000 modelos** de destacados equipos y proveedores de IA, acompañado de vibrantes contribuciones comunitarias y métricas de participación con cientos de miles de usuarios.

- Las librerías de código abierto como Transformers han sido descargadas y destacadas extensamente en GitHub, formando la columna vertebral de muchas innovaciones de IA comerciales y académicas.

- Los clientes empresariales se benefician de niveles de suscripción flexibles — Team (desde $20+/usuario/mes) y Enterprise (precio personalizado), asegurando adopción escalable con seguridad y cumplimiento estándares de la industria.

- Hugging Face mantiene sólidas alianzas con proveedores de infraestructura en la nube y avanza en la eficiencia del cómputo de IA mediante endpoints de inferencia con GPU y hosting ZeroGPU.

---

## 🤝 Talento y Cultura

Hugging Face fomenta un ambiente de trabajo dinámico, inclusivo y orientado a la innovación, comprometido con construir el futuro del aprendizaje automático. Al unirte a su equipo, formas parte de una misión para democratizar la IA mientras colaboras con algunas de las mentes más talentosas en el campo.

- **Aprendizaje y Crecimiento:** Acceso a tecnologías de vanguardia y colaboración directa con la comunidad de IA de código abierto.

- **Flexibilidad:** Roles que apoyan el trabajo remoto y horarios flexibles, facultando a los empleados para equilibrar productividad y bienestar personal.

- **Impacto:** Oportunidad para influir en el desarrollo global de IA en una empresa transparente y centrada en la comunidad.

- **Beneficios:** Compensación competitiva, cobertura médica integral, opciones de acciones y apoyo al desarrollo profesional.

- **Cultura Colaborativa:** Equipos multifuncionales que trabajan abiertamente con énfasis en la experimentación, el aprendizaje y el uso ético de la IA.

Hugging Face recluta activamente a talentos destacados apasionados por la IA y la colaboración abierta. Las vacantes actuales y detalles para postularse están disponibles en https://apply.workable.com/huggingface/.

---

*Para consultas de prensa o asociaciones, contacte a press@huggingface.co*  
*Descubra más y únase a la revolución de la IA en [huggingface.co](https://huggingface.co)*